# ML-Enhanced European Option Pricing: Exploratory Data Analysis

This notebook performs comprehensive exploratory data analysis (EDA) on the processed options data to understand:

1. **Data Distribution**: Understand the distribution of key variables
2. **Black-Scholes Error Analysis**: Examine pricing errors and patterns
3. **Feature Relationships**: Analyze correlations and relationships
4. **Market Microstructure**: Study bid-ask spreads, volume, and liquidity
5. **Outlier Detection**: Identify and analyze outliers
6. **Risk Assessment**: Evaluate assumptions and potential risks

This analysis will inform our modeling approach and feature selection.


## Step 1: Load Processed Data

Load the most recent cleaned options data from the processing pipeline.


In [ ]:
# Load the most recent processed data
import glob
from datetime import datetime

# Find the most recent cleaned data file
processed_files = glob.glob(f"{Config.DATA_PROCESSED_PATH}cleaned_options_data_*.csv")

if not processed_files:
    print("❌ No processed data files found!")
    print(f"Expected location: {Config.DATA_PROCESSED_PATH}")
    print("Please run the data fetching and processing notebook first.")
    raise FileNotFoundError("No processed data available")

# Get the most recent file
latest_file = max(processed_files, key=os.path.getctime)
print(f"📂 Loading data from: {latest_file}")

# Load the data
df = pd.read_csv(latest_file)
df['expiration_date'] = pd.to_datetime(df['expiration_date'])
df['fetch_timestamp'] = pd.to_datetime(df['fetch_timestamp'])

print(f"✅ Loaded {len(df)} options contracts")
print(f"📊 Data shape: {df.shape}")
print(f"📅 Data date range: {df['fetch_timestamp'].min()} to {df['fetch_timestamp'].max()}")

# Display basic info
print(f"\n📋 Dataset Overview:")
print(f"Columns: {list(df.columns)}")
print(f"\nFirst few rows:")
df.head()


## Step 2: Basic Data Summary and Statistics

Examine the basic characteristics and summary statistics of our options dataset.


In [ ]:
# Basic data information
print("🔍 Data Types and Missing Values:")
print("="*50)
print(df.info())

print("\n📈 Summary Statistics:")
print("="*50)
print(df.describe())

print("\n📊 Key Metrics:")
print("="*50)
print(f"• Unique strikes: {df['strike'].nunique()}")
print(f"• Strike range: ${df['strike'].min():.2f} - ${df['strike'].max():.2f}")
print(f"• Expiration dates: {df['expiration_date'].nunique()}")
print(f"• Time to expiry range: {df['time_to_expiry'].min():.4f} - {df['time_to_expiry'].max():.4f} years")
print(f"• Implied volatility range: {df['implied_volatility'].min():.2%} - {df['implied_volatility'].max():.2%}")
print(f"• Market price range: ${df['market_price'].min():.2f} - ${df['market_price'].max():.2f}")
print(f"• Moneyness range: {df['moneyness'].min():.3f} - {df['moneyness'].max():.3f}")

print("\n📋 Missing Values Summary:")
print("="*50)
missing_summary = df.isnull().sum()
if missing_summary.sum() > 0:
    missing_pct = (missing_summary / len(df) * 100).round(2)
    missing_df = pd.DataFrame({
        'Missing Count': missing_summary[missing_summary > 0],
        'Missing %': missing_pct[missing_summary > 0]
    })
    print(missing_df)
else:
    print("✅ No missing values found!")


## Step 3: Black-Scholes Analysis

Calculate theoretical Black-Scholes prices and analyze pricing errors to understand market vs theoretical pricing.


In [ ]:
# Initialize Black-Scholes calculator
bs_calculator = BlackScholesCalculator()
processor = OptionsDataProcessor()

print("🔢 Calculating Black-Scholes theoretical prices...")

# Calculate Black-Scholes prices for all options
bs_prices = []
for _, row in df.iterrows():
    try:
        bs_price = bs_calculator.calculate_option_price(
            S=row['underlying_price'],
            K=row['strike'],
            T=row['time_to_expiry'],
            r=row['risk_free_rate'],
            sigma=row['implied_volatility'],
            option_type='call'
        )
        bs_prices.append(bs_price)
    except Exception as e:
        print(f"Error calculating BS price for row {_}: {e}")
        bs_prices.append(np.nan)

# Add Black-Scholes analysis to dataframe
df_analysis = processor.calculate_target_variable(df, pd.Series(bs_prices))

print(f"✅ Black-Scholes prices calculated for {len(df_analysis)} options")
print(f"\n📊 Pricing Error Analysis:")
print("="*50)
print(f"• Mean pricing error: ${df_analysis['pricing_error'].mean():.4f}")
print(f"• Median pricing error: ${df_analysis['pricing_error'].median():.4f}")
print(f"• Std pricing error: ${df_analysis['pricing_error'].std():.4f}")
print(f"• Min pricing error: ${df_analysis['pricing_error'].min():.4f}")
print(f"• Max pricing error: ${df_analysis['pricing_error'].max():.4f}")

print(f"\n📊 Relative Error Analysis:")
print("="*50)
print(f"• Mean relative error: {df_analysis['relative_error'].mean():.2%}")
print(f"• Median relative error: {df_analysis['relative_error'].median():.2%}")
print(f"• Std relative error: {df_analysis['relative_error'].std():.2%}")

# Show sample with BS prices
print(f"\n🔍 Sample Data with Black-Scholes Prices:")
print("="*50)
sample_cols = ['strike', 'market_price', 'bs_price', 'pricing_error', 'relative_error', 'moneyness', 'implied_volatility']
print(df_analysis[sample_cols].head(10))


## Step 4: Distribution Analysis

Visualize the distributions of key variables to understand data characteristics and identify patterns.


In [ ]:
# Create distribution plots
fig, axes = plt.subplots(3, 3, figsize=(18, 15))
fig.suptitle('Distribution Analysis of Key Variables', fontsize=16, fontweight='bold')

# Variables to plot
variables = [
    ('market_price', 'Market Price ($)'),
    ('bs_price', 'Black-Scholes Price ($)'),
    ('pricing_error', 'Pricing Error ($)'),
    ('relative_error', 'Relative Error (%)'),
    ('implied_volatility', 'Implied Volatility'),
    ('moneyness', 'Moneyness (S/K)'),
    ('time_to_expiry', 'Time to Expiry (years)'),
    ('volume', 'Volume'),
    ('open_interest', 'Open Interest')
]

for i, (var, title) in enumerate(variables):
    row, col = i // 3, i % 3
    ax = axes[row, col]
    
    # Handle missing values for volume
    if var == 'volume':
        data = df_analysis[var].fillna(0)
    elif var == 'relative_error':
        data = df_analysis[var] * 100  # Convert to percentage
    else:
        data = df_analysis[var]
    
    # Create histogram with KDE
    ax.hist(data, bins=20, alpha=0.7, density=True, color='skyblue', edgecolor='black')
    
    # Add KDE if data is not constant
    if data.std() > 0:
        from scipy.stats import gaussian_kde
        kde = gaussian_kde(data.dropna())
        x_range = np.linspace(data.min(), data.max(), 100)
        ax.plot(x_range, kde(x_range), 'r-', linewidth=2, label='KDE')
    
    ax.set_title(title, fontweight='bold')
    ax.set_xlabel(title)
    ax.set_ylabel('Density')
    ax.grid(True, alpha=0.3)
    
    # Add statistics text
    mean_val = data.mean()
    std_val = data.std()
    ax.text(0.05, 0.95, f'Mean: {mean_val:.4f}\\nStd: {std_val:.4f}', 
            transform=ax.transAxes, verticalalignment='top', 
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

# Print distribution statistics
print("📊 Distribution Statistics Summary:")
print("="*70)
for var, title in variables:
    if var == 'volume':
        data = df_analysis[var].fillna(0)
    else:
        data = df_analysis[var]
    
    print(f"{title}:")
    print(f"  Mean: {data.mean():.4f}, Median: {data.median():.4f}, Std: {data.std():.4f}")
    print(f"  Skewness: {stats.skew(data.dropna()):.4f}, Kurtosis: {stats.kurtosis(data.dropna()):.4f}")
    print()


## Step 5: Correlation Analysis

Examine relationships between variables to understand feature dependencies and multicollinearity.


In [ ]:
# Select numerical columns for correlation analysis
numerical_cols = df_analysis.select_dtypes(include=[np.number]).columns.tolist()
# Remove ID-like columns
exclude_cols = ['fetch_timestamp']
numerical_cols = [col for col in numerical_cols if col not in exclude_cols]

# Calculate correlation matrix
correlation_matrix = df_analysis[numerical_cols].corr()

# Create correlation heatmap
plt.figure(figsize=(16, 12))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))  # Mask upper triangle
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='RdBu_r', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8}, fmt='.3f')
plt.title('Correlation Matrix of Options Variables', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Find highly correlated pairs
print("🔍 Highly Correlated Variable Pairs (|correlation| > 0.7):")
print("="*60)
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_val = correlation_matrix.iloc[i, j]
        if abs(corr_val) > 0.7:
            var1 = correlation_matrix.columns[i]
            var2 = correlation_matrix.columns[j]
            high_corr_pairs.append((var1, var2, corr_val))
            print(f"• {var1} ↔ {var2}: {corr_val:.3f}")

if not high_corr_pairs:
    print("✅ No highly correlated pairs found (|correlation| > 0.7)")

# Correlation with target variable (pricing_error)
if 'pricing_error' in correlation_matrix.columns:
    print(f"\n🎯 Correlation with Pricing Error:")
    print("="*50)
    target_corr = correlation_matrix['pricing_error'].sort_values(key=abs, ascending=False)
    for var, corr in target_corr.items():
        if var != 'pricing_error':
            print(f"• {var}: {corr:.4f}")


## Step 6: Market Microstructure Analysis

Analyze bid-ask spreads, volume patterns, and liquidity characteristics of the options.


In [ ]:
# Calculate bid-ask spread metrics
df_analysis['bid_ask_spread'] = df_analysis['ask'] - df_analysis['bid']
df_analysis['relative_spread'] = df_analysis['bid_ask_spread'] / df_analysis['market_price']
df_analysis['mid_price'] = (df_analysis['bid'] + df_analysis['ask']) / 2

print("💰 Market Microstructure Analysis:")
print("="*50)
print(f"• Average bid-ask spread: ${df_analysis['bid_ask_spread'].mean():.4f}")
print(f"• Median bid-ask spread: ${df_analysis['bid_ask_spread'].median():.4f}")
print(f"• Average relative spread: {df_analysis['relative_spread'].mean():.2%}")
print(f"• Median relative spread: {df_analysis['relative_spread'].median():.2%}")

# Volume analysis (handle NaN values)
volume_data = df_analysis['volume'].fillna(0)
print(f"\n📊 Volume Analysis:")
print("="*50)
print(f"• Options with volume data: {df_analysis['volume'].notna().sum()}/{len(df_analysis)}")
print(f"• Average volume (non-zero): {volume_data[volume_data > 0].mean():.1f}")
print(f"• Median volume (non-zero): {volume_data[volume_data > 0].median():.1f}")
print(f"• Total volume: {volume_data.sum():.0f}")

print(f"\n🏦 Open Interest Analysis:")
print("="*50)
print(f"• Average open interest: {df_analysis['open_interest'].mean():.1f}")
print(f"• Median open interest: {df_analysis['open_interest'].median():.1f}")
print(f"• Total open interest: {df_analysis['open_interest'].sum():.0f}")

# Create microstructure visualizations
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Market Microstructure Analysis', fontsize=16, fontweight='bold')

# 1. Bid-Ask Spread vs Moneyness
axes[0,0].scatter(df_analysis['moneyness'], df_analysis['bid_ask_spread'], alpha=0.6, color='blue')
axes[0,0].set_xlabel('Moneyness (S/K)')
axes[0,0].set_ylabel('Bid-Ask Spread ($)')
axes[0,0].set_title('Bid-Ask Spread vs Moneyness')
axes[0,0].grid(True, alpha=0.3)

# 2. Relative Spread vs Moneyness
axes[0,1].scatter(df_analysis['moneyness'], df_analysis['relative_spread']*100, alpha=0.6, color='green')
axes[0,1].set_xlabel('Moneyness (S/K)')
axes[0,1].set_ylabel('Relative Spread (%)')
axes[0,1].set_title('Relative Spread vs Moneyness')
axes[0,1].grid(True, alpha=0.3)

# 3. Volume vs Open Interest
volume_plot = df_analysis['volume'].fillna(0)
axes[0,2].scatter(df_analysis['open_interest'], volume_plot, alpha=0.6, color='red')
axes[0,2].set_xlabel('Open Interest')
axes[0,2].set_ylabel('Volume')
axes[0,2].set_title('Volume vs Open Interest')
axes[0,2].grid(True, alpha=0.3)

# 4. Spread vs Time to Expiry
axes[1,0].scatter(df_analysis['time_to_expiry'], df_analysis['bid_ask_spread'], alpha=0.6, color='purple')
axes[1,0].set_xlabel('Time to Expiry (years)')
axes[1,0].set_ylabel('Bid-Ask Spread ($)')
axes[1,0].set_title('Spread vs Time to Expiry')
axes[1,0].grid(True, alpha=0.3)

# 5. Volume Distribution
volume_nonzero = volume_plot[volume_plot > 0]
if len(volume_nonzero) > 0:
    axes[1,1].hist(volume_nonzero, bins=15, alpha=0.7, color='orange', edgecolor='black')
    axes[1,1].set_xlabel('Volume')
    axes[1,1].set_ylabel('Frequency')
    axes[1,1].set_title('Volume Distribution (Non-Zero)')
    axes[1,1].grid(True, alpha=0.3)
else:
    axes[1,1].text(0.5, 0.5, 'No Volume Data', ha='center', va='center', transform=axes[1,1].transAxes)
    axes[1,1].set_title('Volume Distribution (Non-Zero)')

# 6. Open Interest Distribution
axes[1,2].hist(df_analysis['open_interest'], bins=15, alpha=0.7, color='cyan', edgecolor='black')
axes[1,2].set_xlabel('Open Interest')
axes[1,2].set_ylabel('Frequency')
axes[1,2].set_title('Open Interest Distribution')
axes[1,2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## Step 7: Outlier Detection and Analysis

Identify outliers in the data using multiple methods and analyze their impact on the dataset.


In [ ]:
# Outlier detection using multiple methods
print("🔍 Outlier Detection Analysis:")
print("="*50)

# Method 1: Isolation Forest
clean_data_iso, outliers_iso = processor.detect_outliers(df_analysis, method='isolation_forest', contamination=0.1)
print(f"• Isolation Forest: {len(outliers_iso)} outliers ({len(outliers_iso)/len(df_analysis)*100:.1f}%)")

# Method 2: Z-score method
clean_data_zscore, outliers_zscore = processor.detect_outliers(df_analysis, method='zscore')
print(f"• Z-score method: {len(outliers_zscore)} outliers ({len(outliers_zscore)/len(df_analysis)*100:.1f}%)")

# Method 3: IQR method
clean_data_iqr, outliers_iqr = processor.detect_outliers(df_analysis, method='iqr')
print(f"• IQR method: {len(outliers_iqr)} outliers ({len(outliers_iqr)/len(df_analysis)*100:.1f}%)")

# Analyze outliers in pricing errors specifically
pricing_error_q1 = df_analysis['pricing_error'].quantile(0.25)
pricing_error_q3 = df_analysis['pricing_error'].quantile(0.75)
pricing_error_iqr = pricing_error_q3 - pricing_error_q1
pricing_error_outliers = df_analysis[
    (df_analysis['pricing_error'] < pricing_error_q1 - 1.5 * pricing_error_iqr) |
    (df_analysis['pricing_error'] > pricing_error_q3 + 1.5 * pricing_error_iqr)
]

print(f"\n🎯 Pricing Error Outliers:")
print("="*50)
print(f"• Pricing error outliers: {len(pricing_error_outliers)} ({len(pricing_error_outliers)/len(df_analysis)*100:.1f}%)")
if len(pricing_error_outliers) > 0:
    print(f"• Outlier pricing error range: ${pricing_error_outliers['pricing_error'].min():.4f} to ${pricing_error_outliers['pricing_error'].max():.4f}")
    print(f"• Mean outlier pricing error: ${pricing_error_outliers['pricing_error'].mean():.4f}")

# Create outlier visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Outlier Detection Analysis', fontsize=16, fontweight='bold')

# Box plots for key variables
key_vars = ['pricing_error', 'relative_error', 'implied_volatility', 'market_price', 'moneyness', 'bid_ask_spread']

for i, var in enumerate(key_vars):
    row, col = i // 3, i % 3
    ax = axes[row, col]
    
    # Create box plot
    data_to_plot = df_analysis[var]
    if var == 'relative_error':
        data_to_plot = data_to_plot * 100  # Convert to percentage
        
    box_plot = ax.boxplot(data_to_plot.dropna(), patch_artist=True)
    box_plot['boxes'][0].set_facecolor('lightblue')
    box_plot['boxes'][0].set_alpha(0.7)
    
    ax.set_title(f'{var.replace("_", " ").title()}')
    ax.grid(True, alpha=0.3)
    
    # Add outlier statistics
    q1, q3 = data_to_plot.quantile([0.25, 0.75])
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = data_to_plot[(data_to_plot < lower_bound) | (data_to_plot > upper_bound)]
    
    ax.text(0.02, 0.98, f'Outliers: {len(outliers)}\\n({len(outliers)/len(data_to_plot)*100:.1f}%)', 
            transform=ax.transAxes, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

# Detailed outlier analysis
print(f"\n📊 Detailed Outlier Analysis:")
print("="*50)

if len(pricing_error_outliers) > 0:
    print("Top 5 Pricing Error Outliers:")
    outlier_cols = ['strike', 'market_price', 'bs_price', 'pricing_error', 'moneyness', 'implied_volatility']
    print(pricing_error_outliers[outlier_cols].head().to_string())
    
    print(f"\n🔍 Outlier Characteristics:")
    print(f"• ITM outliers: {(pricing_error_outliers['moneyness'] > 1.0).sum()}")
    print(f"• OTM outliers: {(pricing_error_outliers['moneyness'] < 1.0).sum()}")
    print(f"• High IV outliers (>50%): {(pricing_error_outliers['implied_volatility'] > 0.5).sum()}")
    print(f"• Low time to expiry outliers (<0.01 years): {(pricing_error_outliers['time_to_expiry'] < 0.01).sum()}")
else:
    print("✅ No significant pricing error outliers found using IQR method")


## Step 8: Key Insights and Summary

Summarize the key findings from the exploratory data analysis and their implications for modeling.


In [ ]:
# Generate comprehensive EDA summary
print("📋 EXPLORATORY DATA ANALYSIS SUMMARY")
print("="*60)

print(f"\n📊 DATASET OVERVIEW:")
print(f"• Total options contracts: {len(df_analysis)}")
print(f"• Data completeness: {(1 - df_analysis.isnull().sum().sum() / (df_analysis.shape[0] * df_analysis.shape[1]))*100:.1f}%")
print(f"• Time range: {df_analysis['time_to_expiry'].min():.4f} to {df_analysis['time_to_expiry'].max():.4f} years")
print(f"• Strike range: ${df_analysis['strike'].min():.0f} - ${df_analysis['strike'].max():.0f}")
print(f"• Moneyness range: {df_analysis['moneyness'].min():.3f} - {df_analysis['moneyness'].max():.3f}")

print(f"\n🎯 PRICING ERROR INSIGHTS:")
mean_error = df_analysis['pricing_error'].mean()
std_error = df_analysis['pricing_error'].std()
mean_rel_error = df_analysis['relative_error'].mean()
print(f"• Mean absolute pricing error: ${abs(mean_error):.4f}")
print(f"• Pricing error volatility: ${std_error:.4f}")
print(f"• Mean relative error: {mean_rel_error:.2%}")
print(f"• Error direction: {'Market overpriced' if mean_error > 0 else 'Market underpriced'} vs Black-Scholes")

print(f"\n💰 MARKET MICROSTRUCTURE:")
print(f"• Average bid-ask spread: ${df_analysis['bid_ask_spread'].mean():.4f}")
print(f"• Average relative spread: {df_analysis['relative_spread'].mean():.2%}")
print(f"• Volume data availability: {df_analysis['volume'].notna().sum()}/{len(df_analysis)} contracts")
print(f"• Liquidity concentration: Top 10% OI contracts hold {df_analysis.nlargest(int(len(df_analysis)*0.1), 'open_interest')['open_interest'].sum() / df_analysis['open_interest'].sum() * 100:.1f}% of total OI")

print(f"\n🔍 DATA QUALITY ASSESSMENT:")
# Count extreme values
extreme_iv = (df_analysis['implied_volatility'] > 1.0).sum()
extreme_moneyness = ((df_analysis['moneyness'] < 0.5) | (df_analysis['moneyness'] > 2.0)).sum()
short_expiry = (df_analysis['time_to_expiry'] < 0.01).sum()

print(f"• High implied volatility (>100%): {extreme_iv} contracts")
print(f"• Extreme moneyness (<0.5 or >2.0): {extreme_moneyness} contracts")
print(f"• Very short expiry (<0.01 years): {short_expiry} contracts")
print(f"• Outlier rate (Isolation Forest): {len(outliers_iso)/len(df_analysis)*100:.1f}%")

print(f"\n📈 MODELING IMPLICATIONS:")
print("="*60)

# Check for high correlations that might cause multicollinearity
high_corr_count = len(high_corr_pairs)
print(f"• Multicollinearity risk: {high_corr_count} highly correlated pairs found")

# Distribution characteristics
skewness_issues = []
for var in ['pricing_error', 'relative_error', 'implied_volatility']:
    skew_val = stats.skew(df_analysis[var].dropna())
    if abs(skew_val) > 1:
        skewness_issues.append(f"{var} (skew: {skew_val:.2f})")

if skewness_issues:
    print(f"• Distribution concerns: {', '.join(skewness_issues)}")
else:
    print("• Distribution quality: Good - no severe skewness detected")

# Feature engineering recommendations
print(f"\n🛠️ RECOMMENDED PREPROCESSING:")
print("• Handle missing volume data (imputation or indicator variables)")
print("• Consider log transformation for skewed variables")
if high_corr_count > 0:
    print("• Apply dimensionality reduction or feature selection for correlated variables")
print("• Robust scaling recommended due to outliers")
print("• Consider separate models for different moneyness regimes")

print(f"\n✅ DATASET READINESS:")
data_quality_score = (
    (1 - df_analysis.isnull().sum().sum() / (df_analysis.shape[0] * df_analysis.shape[1])) * 30 +  # Completeness
    (1 - len(outliers_iso)/len(df_analysis)) * 30 +  # Outlier rate
    (1 - min(abs(mean_rel_error), 0.5) / 0.5) * 20 +  # Pricing error magnitude
    (1 - min(high_corr_count, 10) / 10) * 20  # Multicollinearity
) 
print(f"Overall data quality score: {data_quality_score:.1f}/100")

if data_quality_score >= 80:
    print("🟢 EXCELLENT - Dataset ready for modeling")
elif data_quality_score >= 60:
    print("🟡 GOOD - Minor preprocessing needed")
else:
    print("🔴 NEEDS WORK - Significant data issues to address")

print(f"\n📁 Analysis complete! Processed data available at:")
print(f"   {latest_file}")


In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

sys.path.append('..')

from config import Config
from src.black_scholes import BlackScholesCalculator
from src.data_processor import OptionsDataProcessor

plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("Ready for EDA")
